In [1]:
import itertools
import os
import re
import math
import random
import numpy as np
import matplotlib.pyplot as plt
import csv
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

from sklearn import metrics
from sklearn.linear_model import SGDClassifier

from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier,     RandomForestRegressor, RandomForestClassifier
from datetime import date

In [2]:
print('loading the data...')
df = pd.read_csv('df_data.csv')
donation = pd.read_csv('donation_data.csv')
resource = pd.read_csv('resource_data.csv')
essay = pd.read_csv('essay_data.csv')
print('complete..')

# fisrt partation data into several parts and train and generate based on those pieses
#part_1 = example[(example['date_posted']<"2011-01-01")&(example['date_posted']>="2010-01-01")]
#part_2 = example[(example['date_posted']<"2012-01-01")&(example['date_posted']>="2011-01-01")]
#part_3 = example[(example['date_posted']<"2013-01-01")&(example['date_posted']>="2012-01-01")]
#part_4 = example[(example['date_posted']<"2014-01-01")&(example['date_posted']>="2013-01-01")]

loading the data...
complete..


In [3]:
print(df.shape)
print(donation.shape)
print(resource.shape)
print(essay.shape)

(467620, 101)
(467620, 10)
(467620, 13)
(467620, 6)


In [4]:

df.drop('Unnamed: 0',axis=1,inplace=True)
donation.drop('Unnamed: 0',axis=1,inplace=True)
donation.drop('schoolid',axis=1,inplace=True)
donation.drop('teacher_acctid',axis=1,inplace=True)
resource.drop('Unnamed: 0',axis=1,inplace=True)
essay.drop('Unnamed: 0',axis=1,inplace=True)
print(df.columns)
print(donation.columns)
print(resource.columns)
print(essay.columns)
df.head()

Index(['projectid', 'teacher_acctid', 'schoolid', 'school_ncesid',
       'school_latitude', 'school_longitude', 'school_city', 'school_state',
       'school_zip', 'school_metro', 'school_district', 'school_county',
       'school_charter', 'school_magnet', 'school_year_round', 'school_nlns',
       'school_kipp', 'school_charter_ready_promise', 'teacher_prefix',
       'teacher_teach_for_america', 'teacher_ny_teaching_fellow',
       'primary_focus_subject', 'primary_focus_area',
       'secondary_focus_subject', 'secondary_focus_area', 'resource_type',
       'poverty_level', 'grade_level', 'fulfillment_labor_materials',
       'total_price_excluding_optional_support',
       'total_price_including_optional_support', 'students_reached',
       'eligible_double_your_impact_match', 'eligible_almost_home_match',
       'date_posted', 'is_exciting', 'at_least_1_teacher_referred_donor',
       'fully_funded', 'at_least_1_green_donation', 'great_chat',
       'three_or_more_non_teacher_re

,projectid,teacher_acctid,schoolid,school_ncesid,school_latitude,school_longitude,school_city,school_state,school_zip,school_metro,...,school_county_great_chat,school_county_one_or_more_required,school_county_count,school_zip_is_exciting,school_zip_at_least_1_teacher_referred_donor,school_zip_fully_funded,school_zip_at_least_1_green_donation,school_zip_great_chat,school_zip_one_or_more_required,school_zip_count
0,00001ccc0e81598c4bd86bacb94d7acb,96963218e74e10c3764a5cfb153e6fea,9f3f9f2c2da7edda5648ccd10554ed8c,1.709930e+11,41.807654,-87.673257,Chicago,IL,60609.0,urban,...,0.371600,0.699309,15694,0.085692,0.274297,0.673771,0.657226,0.295037,0.702203,769
1,00002bff514104264a6b798356fdd893,3414541eb63108700b188648f866f483,cbaae3265eda78d330cb8ab1a9217071,6.032700e+10,35.203447,-118.840956,Arvin,CA,93203.0,NaN,...,0.203624,0.720264,4187,0.017430,0.086567,0.511819,0.360627,0.207809,0.538079,148
2,00002d691c05c51a5fdfbb2baef0ba25,7ad6abc974dd8b62773f79f6cbed48d5,56502bae9e97bab5eb54f9001878f469,6.029700e+10,34.137997,-118.062795,Arcadia,CA,91007.0,urban,...,0.219667,0.697394,41797,0.034273,0.121711,0.539425,0.499051,0.356082,0.637111,12
3,0000b38bbc7252972f7984848cf58098,e1aa1ae5301d0cda860c4d9c89c24919,30fcfca739b17be54ce3f1ee46980340,2.311400e+11,44.437717,-70.201292,Livermore,ME,4253.0,rural,...,0.438209,0.647539,498,0.053858,0.262688,0.776239,0.641367,0.273844,0.715460,4
4,0000ee613c92ddc5298bf63142996a5c,e0c0a0214d3c2cfdc0ab6639bc3c5342,38bb0d62aa613c2f933de56c9df855b7,5.101260e+11,38.851982,-77.145287,Falls Church,VA,22041.0,suburban,...,0.379813,0.699439,792,0.129449,0.333712,0.685081,0.619970,0.339097,0.655739,19


In [5]:

essay.drop('schoolid',axis=1,inplace=True)
essay.drop('teacher_acctid',axis=1,inplace=True)
data = df
data = pd.merge(df,donation,how='left',on='projectid')
print(data.shape)

(467620, 106)


In [6]:
resource.drop('students_reached',axis=1,inplace=True)
data = pd.merge(data,resource,how='left',on='projectid')
data = pd.merge(data,essay,how='left',on='projectid')

In [7]:
data['date_posted'] = pd.to_datetime(data['date_posted'])
ref_date = "2010-01-01"
ref_date = pd.to_datetime(ref_date)
print(ref_date)
data['daysbet'] = data['date_posted'] - ref_date
data['daysbet'] = data['daysbet'].dt.days
data['monthbet'] = data['date_posted'] - ref_date
data['monthbet'] = data['monthbet'].dt.days
data['monthbet'] = data['monthbet']/30

2010-01-01 00:00:00


In [8]:
print(data.shape)
data.head()

(467620, 120)


,projectid,teacher_acctid,schoolid,school_ncesid,school_latitude,school_longitude,school_city,school_state,school_zip,school_metro,...,Supplies_cost,Other_cost,Books_cost,Trips_cost,Visitors_cost,price_per_student,essay_length,title_length,daysbet,monthbet
0,00001ccc0e81598c4bd86bacb94d7acb,96963218e74e10c3764a5cfb153e6fea,9f3f9f2c2da7edda5648ccd10554ed8c,1.709930e+11,41.807654,-87.673257,Chicago,IL,60609.0,urban,...,1225.44,0.0,0.0,0.0,0.0,39.530323,1453,25,1199,39.966667
1,00002bff514104264a6b798356fdd893,3414541eb63108700b188648f866f483,cbaae3265eda78d330cb8ab1a9217071,6.032700e+10,35.203447,-118.840956,Arvin,CA,93203.0,NaN,...,399.74,0.0,0.0,0.0,0.0,19.987000,1698,25,250,8.333333
2,00002d691c05c51a5fdfbb2baef0ba25,7ad6abc974dd8b62773f79f6cbed48d5,56502bae9e97bab5eb54f9001878f469,6.029700e+10,34.137997,-118.062795,Arcadia,CA,91007.0,urban,...,0.00,0.0,774.8,0.0,0.0,3.099200,1273,18,343,11.433333
3,0000b38bbc7252972f7984848cf58098,e1aa1ae5301d0cda860c4d9c89c24919,30fcfca739b17be54ce3f1ee46980340,2.311400e+11,44.437717,-70.201292,Livermore,ME,4253.0,rural,...,0.00,0.0,0.0,0.0,0.0,12.347500,2036,42,1365,45.500000
4,0000ee613c92ddc5298bf63142996a5c,e0c0a0214d3c2cfdc0ab6639bc3c5342,38bb0d62aa613c2f933de56c9df855b7,5.101260e+11,38.851982,-77.145287,Falls Church,VA,22041.0,suburban,...,0.00,0.0,0.0,0.0,0.0,17.778421,2375,47,1440,48.000000


In [9]:
# start to calculate history rate for different attributes 
id_columns = ['teacher_acctid','schoolid']
location_columns = ['school_city','school_state','school_district','school_county','school_zip']
all_columns = id_columns + location_columns
one_or_more_required = ['three_or_more_non_teacher_referred_donors','one_non_teacher_referred_donor_giving_100_plus',
                       'donation_from_thoughtful_donor']
requirements_columns = ['projectid','date_posted','is_exciting','at_least_1_teacher_referred_donor','fully_funded','at_least_1_green_donation','great_chat','one_or_more_required']
require_only_columns = ['is_exciting','at_least_1_teacher_referred_donor','fully_funded','at_least_1_green_donation','great_chat','one_or_more_required']

date_columns = ['day','month','year']

projects = pd.read_csv('./data/projects.csv')
sub_primary_requirements = ['great_messages_proportion','teacher_referred_count','non_teacher_referred_count'] 
projects_numeric_columns = ['school_latitude','school_longitude','fulfillment_labor_materials',
                           'total_price_excluding_optional_support',
                           'total_price_including_optional_support','grade_level','poverty_level','students_reached']
projects_id_columns =['projectid','teacher_acctid','schoolid','school_ncesid']
projects_categorial_columns = projects.columns - projects_numeric_columns - projects_id_columns - ['date_posted']

feature_columns = data.columns - data.columns[0:4] - requirements_columns-['cat'] - one_or_more_required-sub_primary_requirements

/Users/hang/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:18: FutureWarning: using '-' to provide set differences with Indexes is deprecated, use .difference()
/Users/hang/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:20: FutureWarning: using '-' to provide set differences with Indexes is deprecated, use .difference()


In [10]:
data.fillna(method='pad',inplace=True)

(467620, 120)

In [12]:
for var in projects_categorial_columns:
    le = LabelEncoder()
    data[var] = le.fit_transform(data[var])

,projectid,teacher_acctid,schoolid,school_ncesid,school_latitude,school_longitude,school_city,school_state,school_zip,school_metro,...,Supplies_cost,Other_cost,Books_cost,Trips_cost,Visitors_cost,price_per_student,essay_length,title_length,daysbet,monthbet
467615,fffeebf4827d745aa36b17c2d38d1966,96c1a33b4f2b832595c3c6a01bc24c05,7bb6b1924a7bd181ab5568b062260966,6.346201e+10,38.654224,-121.308122,2361,4,14731,1,...,0.00,693.45,0.0,0.0,0.0,63.040909,1212,16,1065,35.500000
467616,ffff2d9c769c8fb5335e949c615425eb,485784e52a4c85ea6783285d09d8aea4,f8e49225a821cc04e1176303e55d89ec,1.602640e+11,42.820593,-112.409161,5964,13,12935,2,...,0.00,0.00,0.0,0.0,0.0,19.999600,2799,21,1157,38.566667
467617,ffff7266778f71242675416e600b94e1,b9a8f14199e0d8109200ece179281f4f,9655f475102286a53b98ffbabb14013f,3.703780e+11,35.830690,-79.865901,7003,27,3472,0,...,0.00,131.25,0.0,0.0,0.0,5.468750,1074,15,1557,51.900000
467618,ffff97ed93720407d70a2787475932b0,462270f5d5c212162fcab11afa2623cb,9943fdb43ab5f2b53f27e961caf9e90a,3.600153e+11,40.631962,-73.923897,859,34,1414,2,...,399.02,0.00,0.0,0.0,0.0,15.960800,1327,44,253,8.433333
467619,ffffac55ee02a49d1abc87ba6fc61135,947066d0af47e0566f334566553dd6a6,103ed937d8d84fdffe8b87a4811f00bf,1.301230e+11,33.514296,-84.345336,3624,10,4324,1,...,0.00,0.00,0.0,0.0,0.0,18.500000,1155,25,526,17.533333


In [13]:
# load data
train = data[(data['cat']=='train')|(data['cat']=='val')]
feature_data = train[feature_columns]
X = feature_data
names = feature_data.columns.values
Y = train['is_exciting']

test = data[(data['cat']=='test')]
X_test = test[feature_columns]
Y_test = test['is_exciting']

In [16]:
# after get full df, do feature selection 
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier

rfr = RandomForestClassifier(n_estimators=300,n_jobs=-1,verbose=1,max_depth=5,max_features='log2',min_samples_split=1)
print("start to fit the data")
y_ = Y==1
rfr.fit(X,y_)
print("finish fitting")
print( "Features sorted by their score:")
feature_list = sorted(zip(map(lambda x: round(x, 4), rfr.feature_importances_), names), 
             reverse=True)
print(feature_list)

start to fit the data


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   18.2s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:  1.2min


finish fitting
Features sorted by their score:
[(0.31790000000000002, 'teacher_acctid_is_exciting'), (0.14799999999999999, 'schoolid_is_exciting'), (0.090300000000000005, 'teacher_acctid_at_least_1_teacher_referred_donor'), (0.053199999999999997, 'schoolid_at_least_1_teacher_referred_donor'), (0.052499999999999998, 'teacher_acctid_great_chat'), (0.0465, 'school_zip_is_exciting'), (0.027400000000000001, 'schoolid_great_chat'), (0.0264, 'teacher_acctid_at_least_1_green_donation'), (0.022499999999999999, 'school_district_is_exciting'), (0.019199999999999998, 'school_city_is_exciting'), (0.012999999999999999, 'daysbet'), (0.012699999999999999, 'school_zip_great_chat'), (0.011299999999999999, 'teacher_acctid_fully_funded'), (0.0112, 'monthbet'), (0.010999999999999999, 'school_zip_at_least_1_teacher_referred_donor'), (0.0106, 'schoolid_at_least_1_green_donation'), (0.0097000000000000003, 'teacher_acctid_count'), (0.0091000000000000004, 'teacher_acctid_one_or_more_required'), (0.0085000000000

[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  1.9min finished


In [18]:
rfr_predict = rfr.predict_proba(X_test)
rfr_predict = rfr_predict[:,1]
sample = pd.read_csv('./data/sampleSubmission.csv')
sample = sample.sort_values(by='projectid')
sample['is_exciting'] = rfr_predict
sample.to_csv('predictions_rf_300_4_log2_1.csv', index = False)

[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.5s
[Parallel(n_jobs=4)]: Done 300 out of 300 | elapsed:    0.8s finished


In [56]:
from sklearn.naive_bayes import BernoulliNB
nb = BernoulliNB()
print("start training")
nb.fit(X,y_)
print("finsh training")

start training
finsh training


In [57]:
nb_predict = nb.predict_proba(X_test)
nb_predict = nb_predict[:,1]
sample = pd.read_csv('./data/sampleSubmission.csv')
sample = sample.sort_values(by='projectid')
sample['is_exciting'] = nb_predict
sample.to_csv('predictions_nb.csv', index = False)

In [44]:
rfr_predict

array([ 0.34 ,  0.16 ,  0.015, ...,  0.01 ,  0.405,  0.045])

In [41]:
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier
dctr = DecisionTreeClassifier()

print("start to fit the data")
y_ = Y==1
dctr.fit(X,Y)
print("finish fitting")
dctr_predict = dctr.predict_proba(X_test)
dctr_predict = dctr_predict[:,1]
print( "Features sorted by their score:")
feature_list_2 = sorted(zip(map(lambda x: round(x, 4), dctr.feature_importances_), names), 
             reverse=True)
print(feature_list_2)
sample = pd.read_csv('./data/sampleSubmission.csv')
sample = sample.sort_values(by='projectid')
sample['is_exciting'] = dctr_predict
sample.to_csv('predictions_dt.csv', index = False)

start to fit the data
finish fitting
Features sorted by their score:
[(0.21190000000000001, 'teacher_acctid_is_exciting'), (0.040000000000000001, 'essay_length'), (0.036799999999999999, 'schoolid_is_exciting'), (0.0339, 'title_length'), (0.033500000000000002, 'avg_cost'), (0.030599999999999999, 'daysbet'), (0.029899999999999999, 'monthbet'), (0.028400000000000002, 'day'), (0.027799999999999998, 'price_per_student'), (0.026599999999999999, 'teacher_acctid_one_or_more_required'), (0.024799999999999999, 'cost'), (0.0218, 'secondary_focus_subject'), (0.019699999999999999, 'total_price_including_optional_support'), (0.019, 'total_price_excluding_optional_support'), (0.019, 'teacher_acctid_count_donor'), (0.0189, 'item_quantity'), (0.016799999999999999, 'students_reached'), (0.0161, 'teacher_acctid_donor_donation_total_donor'), (0.015800000000000002, 'primary_focus_subject'), (0.015699999999999999, 'school_latitude'), (0.0152, 'school_zip_count'), (0.0152, 'school_zip'), (0.01420000000000000

In [45]:
# after feature selection, select to drop some columns
update_feature_columns = []
for (v,n) in feature_list:
    update_feature_columns.append(n)
update_feature_columns_2 = []
for (v,n) in feature_list_2:
    update_feature_columns_2.append(n)
used_feature = list(set(update_feature_columns[0:40]).union(set(update_feature_columns_2[0:40])))

In [46]:
# load data again
train = data[(data['cat']=='train')|(data['cat']=='val')]
feature_data = train[used_feature]
X = feature_data
names = feature_data.columns.values
Y = train['is_exciting']

test = data[(data['cat']=='test')]
X_test = test[used_feature]
Y_test = test['is_exciting']

In [47]:
#from sklearn.ensemble import AdaBoostRegressor,AdaBoostClassifier
#adb = AdaBoostClassifier(n_estimators=100,learning_rate=0.1)
#adb.fit(X,Y)
#predict = adb.predict(X_test)

In [48]:
from sklearn.ensemble import GradientBoostingClassifier,GradientBoostingRegressor
y_ = Y==1
# Fit regression model
params = {'n_estimators': 100, 'max_depth': 7, 'min_samples_split': 1,
          'learning_rate': 0.1, 'loss': 'deviance','verbose':1,}
clf = GradientBoostingClassifier(**params)
print ("start training")
clf.fit(X, Y)
print("finish training")

start training
      Iter       Train Loss   Remaining Time 
         1           0.5324           32.07m
         2           0.4991           31.33m
         3           0.4745           32.30m
         4           0.4552           31.67m
         5           0.4396           30.99m
         6           0.4266           30.47m
         7           0.4156           29.55m
         8           0.4062           28.95m
         9           0.3981           28.60m
        10           0.3910           28.38m
        20           0.3522           25.92m
        30           0.3365           24.06m
        40           0.3289           20.12m
        50           0.3241           16.51m
        60           0.3208           15.76m
        70           0.3183           18.63m
        80           0.3159           11.62m
        90           0.3139            5.48m
       100           0.3119            0.00s
finish training


In [58]:
#predict = adb.predict(X_test)
predict = clf.predict_proba(X_test)
predict = predict[:,1]

In [59]:
predict

array([ 0.05956156,  0.18716356,  0.00057773, ...,  0.00061795,
        0.2962276 ,  0.00451227])

In [60]:
sample = pd.read_csv('./data/sampleSubmission.csv')
sample = sample.sort_values(by='projectid')
sample['is_exciting'] = predict
sample.head()

,projectid,is_exciting
44771,00034e54ed99042609edad55031c8861,0.059562
44770,00057f424b5498c7ece13d13ce3e2178,0.187164
44769,00059e63bb0567708b2b0c9e3d9c43d6,0.000578
44768,0008c67f27dd29ea7be5a7cc5a866df8,0.479150
44767,000b6e707ad50a597ab46eedff6bde05,0.001128


In [61]:
sample.to_csv('predictions_4.csv', index = False)